In [2]:
import random
import csv
import os
import os.path
import shutil
import cv2

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm # Displays a progress bar

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader, random_split

import pandas as pd
import h5py

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

!gdown https://drive.google.com/uc?id=1olKESqgAm5GeAmuRC21BpTBkxFrgmUIp
!unzip /content/Data

Downloading...
From: https://drive.google.com/uc?id=1olKESqgAm5GeAmuRC21BpTBkxFrgmUIp
To: /content/Data.zip
977MB [00:04, 210MB/s]
Archive:  /content/Data.zip
   creating: Data/Test/
   creating: Data/Test/Images/
  inflating: Data/Test/Images/imagedata.hdf5  
   creating: Data/Test/Labels/
  inflating: Data/Test/Labels/labeldata.hdf5  
   creating: Data/Train/
   creating: Data/Train/Images/
  inflating: Data/Train/Images/imagedata.hdf5  
   creating: Data/Train/Labels/
  inflating: Data/Train/Labels/labeldata.hdf5  


In [4]:
class DataNonNorm:
    def __init__(self, root):
        self.ROOT = root
        self.images = self.read_images(os.path.join(root, "Images"))
        self.labels = self.read_labels(os.path.join(root, "Labels"))

    def __len__(self):
        # Return number of points in the dataset
        return len(self.images)

    def __getitem__(self, idx):
        # Here we have to return the item requested by `idx`. The PyTorch DataLoader class will use this method to make an iterable for training/validation loop.
        img = self.images[idx] / 255
        #label = self.labels[idx]
        return img
  
    # Read Images
    def read_images(self, path:str) -> list:
        imgs = []
        with h5py.File(os.path.join(path, "imagedata.hdf5"), "r") as f:
            for i in sorted(f.keys(), key = int ):
                imgs.append(np.array(f.get(str(i))))
        #output = torch.tensor(output)
        return imgs
      
    # Read Labels
    def read_labels(self, path:str) -> list:
        output = []
        with h5py.File(os.path.join(path, "labeldata.hdf5"), "r") as f:
            labels = f["labels"][()]
        return labels       

# # Load the dataset and train and test splits
# print("Loading datasets...")

# # Data path
dataTrain = DataNonNorm(os.path.normpath('./Data/Train'))
dataTest  = DataNonNorm(os.path.normpath('./Data/Test'))

In [5]:
trainloader = DataLoader(dataTrain, batch_size=32, shuffle=True)
testloader = DataLoader(dataTest, batch_size=2, shuffle=True)

In [6]:
import math
# Get mean and std deviation of training and test
tnMean = 0.
tnStd = 0.
count = 0.
var = 0.
for data in trainloader:
    tnMean += np.mean(np.array(data))
    var += np.var(np.array(data))
    count += 1

tnMean /= count
tnStd = math.sqrt(var / count)

print(tnMean)
print(tnStd)

tstMean = 0.
tstStd = 0.
count = 0.
var = 0.
for data in testloader:
    tstMean += np.mean(np.array(data))
    var += np.var(np.array(data))
    count += 1

tstMean /= count
tstStd = math.sqrt(var / count)

print(tstMean)
print(tstStd)

0.6535338665393978
0.2691491382000663
0.6536001114379154
0.26438978120602336


In [27]:
params = {
  "tnMean": tnMean,
  "tstMean": tstMean,
  "tnStd": tnStd,
  "tstStd": tstStd,
  "batch": 32,
  "l1Tol2Features": 25,
  "epochs": 30,
  "lr": 0.001,
  "wtDecay": 0.004,
  "frozenChildren": 5
}

In [10]:
class Data:

    def __init__(self, root, mn, std):
        self.ROOT = root
        self.images = self.read_images(os.path.join(root, "Images"))
        self.labels = self.read_labels(os.path.join(root, "Labels"))

        self.transform = MyTransform = transforms.Compose([
            transforms.ToTensor(), # Transform from [0,255] uint8 to [0,1] float
            transforms.Normalize((mn, mn, mn), (std, std, std)) # TODO: Normalize to zero mean and unit variance with appropriate parameters 0.5
            ])

    def __len__(self):
        # Return number of points in the dataset
        return len(self.images)

    def __getitem__(self, idx):
        # Here we have to return the item requested by `idx`. The PyTorch DataLoader class will use this method to make an iterable for training/validation loop.
        img = self.transform(self.images[idx])
        # img = self.images[idx] / 255
        label = self.labels[idx]
        return img, label
  
    # Read Images
    def read_images(self, path:str) -> list:
        imgs = []
        with h5py.File(os.path.join(path, "imagedata.hdf5"), "r") as f:
            for i in sorted(f.keys(), key = int ):
                imgs.append(np.array(f.get(str(i))))
        #output = torch.tensor(output)
        return imgs
      
    # Read Labels
    def read_labels(self, path:str) -> list:
        output = []
        with h5py.File(os.path.join(path, "labeldata.hdf5"), "r") as f:
            labels = f["labels"][()]
        return labels       

# Load the dataset and train and test splits
print("Loading datasets...")

# Data path
dataTrain = Data(os.path.normpath('./Data/Train'), tnMean, tnStd)
dataTest  = Data(os.path.normpath('./Data/Test'),tstMean, tstStd)

Loading datasets...


In [ ]:
print(dataTrain.__len__())
print(len(dataTrain.labels))

In [ ]:
from PIL import Image

with h5py.File(os.path.join('./Data/Train/Images', "imagedata.hdf5"), "r") as f:
  img = Image.fromarray(np.array(f.get("1")))
display(img)

In [13]:
# print("Done!")

# # Create dataloaders
# # TODO: Experiment with different batch sizes
trainloader = DataLoader(dataTrain, batch_size=params["batch"], shuffle=True, drop_last=True)
testloader = DataLoader(dataTest, batch_size=32, shuffle=True, drop_last=True)

In [14]:
from torchvision import datasets, transforms
import torchvision.models as models

In [30]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # TODO: [Transfer learning with pre-trained ResNet-50] 1) Define how many first layers of convolutoinal neural network (CNN) feature extractor in ResNet-50 to be "frozen" and 2) design your own fully-connected network (FCN) classifier.
        # 1) You will only refine last several layers of CNN feature extractor in ResNet-50 that mainly relate to high-level vision task. Determine how many first layers of ResNet-50 should be frozen to achieve best performances. Commented codes below will help you understand the architecture, i.e., "children", of ResNet-50.
        # 2) Design your own FCN classifier. Here I provide a sample of two-layer FCN.
        # Refer to PyTorch documentations of torch.nn to pick your layers. (https://pytorch.org/docs/stable/nn.html)
        # Some common Choices are: Linear, ReLU, Dropout, MaxPool2d, AvgPool2d
        # If you have many layers, consider using nn.Sequential() to simplify your code
        
        # Load pretrained ResNet-50
        self.model_resnet = models.resnet50(pretrained=True)
        
        # The code below can show children of ResNet-50
        # child_counter = 0
        # for child in model.children():
        #    print(" child", child_counter, "is -")
        #    print(child)
        #    child_counter += 1
        
        # TODO: Determine how many first layers of ResNet-50 to freeze
        child_counter = 0
        for child in self.model_resnet.children():
            if child_counter < params['frozenChildren']:
                for param in child.parameters():
                    param.requires_grad = False
            elif child_counter == params['frozenChildren']:
                children_of_child_counter = 0
                for children_of_child in child.children():
                    if children_of_child_counter < 3:
                        for param in children_of_child.parameters():
                            param.requires_grad = False
                    else:
                        children_of_child_counter += 1
            else:
                print("child ",child_counter," was not frozen")
            child_counter += 1
        
        # Set ResNet-50's FCN as an identity mapping
        num_fc_in = self.model_resnet.fc.in_features
        self.model_resnet.fc = nn.Identity()
        
        # TODO: Design your own FCN
        self.fc1 = nn.Linear(num_fc_in, params['l1Tol2Features'], bias = 3) # from input of size num_fc_in to output of size ?
        self.bn1 = nn.BatchNorm1d(num_features=params['l1Tol2Features'])
        self.fc2 = nn.Linear(params['l1Tol2Features'], 3, bias = 3) # from hidden layer to 3 class scores

    def forward(self,x):
        # TODO: Design your own network, implement forward pass here
        
        relu = nn.ReLU() # No need to define self.relu because it contains no parameters
        
        with torch.no_grad():
            features = self.model_resnet(x)
            
        x = self.fc1(features) # Activation are flattened before being passed to the fully connected layers
        x = self.bn1(x)
        x = relu(x)
        x = self.fc2(x)
        
        # The loss layer will be applied outside Network class
        return x

device = "cuda" if torch.cuda.is_available() else "cpu" # Configure device
model= Network().to(device)
criterion = nn.CrossEntropyLoss() # Specify the loss layer (note: CrossEntropyLoss already includes LogSoftMax())
# TODO: Modify the line below, experiment with different optimizers and parameters (such as learning rate)
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()), 
           lr = params['lr'], 
           weight_decay = params['wtDecay'] # Specify optimizer and assign trainable parameters to it, weight_decay is L2 regularization strength (default: lr=1e-2, weight_decay=1e-4)
)

num_epochs = params['epochs'] # TODO: Choose an appropriate number of training epochs
saveData = []

def train(model, loader, num_epoch = num_epochs): # Train the model
    print("Start training...")
    model.train() # Set the model to training mode
    for i in range(num_epoch):
        running_loss = []
        for batch, label in tqdm(loader, position=0, leave=True):
            batch = batch.to(device)
            label = label.to(device)
            optimizer.zero_grad() # Clear gradients from the previous iteration
            pred = model(batch) # This will call Network.forward() that you implement
            loss = criterion(pred, label) # Calculate the loss
            running_loss.append(loss.item())
            loss.backward() # Backprop gradients to all tensors in the network
            optimizer.step() # Update trainable weights
        print("Epoch {} loss:{}".format(i+1,np.mean(running_loss))) # Print the average loss for this epoch
        saveData.append((i+1, np.mean(running_loss)))
    print("Done!")

def evaluate(model, loader): # Evaluate accuracy on validation / test set
    model.eval() # Set the model to evaluation mode
    correct = 0
    with torch.no_grad(): # Do not calculate grident to speed up computation
        for batch, label in tqdm(loader, position=0, leave=True):
            batch = batch.to(device)
            label = label.to(device)
            pred = model(batch)
            correct += (torch.argmax(pred,dim=1)==label).sum().item()
    acc = correct/len(loader.dataset)
    print("Evaluation accuracy: {}".format(acc))
    return acc
    
train(model, trainloader, num_epochs)
print("Evaluate on test set")
accuracy = evaluate(model, testloader)

  0%|          | 0/420 [00:00<?, ?it/s]

child  6  was not frozen
child  7  was not frozen
child  8  was not frozen
child  9  was not frozen
Start training...


  0%|          | 1/420 [00:00<00:54,  7.63it/s]

Epoch 1 loss:0.6355305328965187


  0%|          | 1/420 [00:00<00:55,  7.58it/s]

...

Epoch 29 loss:0.11222827444961737


  1%|          | 1/101 [00:00<00:12,  8.11it/s]

Epoch 30 loss:0.11924057688031878
Done!
Evaluate on test set


100%|██████████| 101/101 [00:12<00:00,  7.96it/s]

Evaluation accuracy: 0.8609312365094048


In [39]:
import csv
accuracy = 0.8741905642923219
with open('/content/params.csv', 'w+') as store:
  storeWrite = csv.writer(store, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  #storeWrite.writerow(["tnMean", "tstMean", "tnStd", "tstStd", "batch", "l1Tol2Features", "epochs", "lr", "wtDecay", "frozenChildren"])
  for key, val in params.items():
    storeWrite.writerow([key, val])
  storeWrite.writerow(['epochNum', 'loss'])
  storeWrite.writerows(saveData)
  storeWrite.writerow(['Accuracy', accuracy])
